In [4]:
import json, sys, settings, random
import lasair
import math, numpy as np

## The `jumpFromMean` feature
This simple feature is to filter out lightcurves with a significant change in brightness. 
It lumps together all 6 wavebands, computes mean and standard deviation, then number of sigma for latest diaSource.

In [5]:
max_alert    = 100

In [6]:
# the lasair client will be used for pulling all the info about the object
# and for annotating it.
# If you want to use the cache feature, just 'mkdir cache' in the current directory
L = lasair.lasair_client(settings.API_TOKEN, endpoint='https://lasair-lsst.lsst.ac.uk/api')
selected = 'objects.diaObjectId, sherlock_classifications.classification as class'
tables = 'objects,sherlock_classifications'
conditions = 'nSources>20'

results = L.query(selected, tables, conditions, limit=max_alert)

### This function looks for number of sigma change
Gets mean and sigma from old diaSources, then computes number fo sigma for latest diaSource

In [7]:
def sigmaJump(alert, timeInterval, nLimit):
    # for ZTF, the candidates and noncandiates are mixed in, and the noncandidates have no candid
    sources  = [c for c in alert['candidates'] if 'candid' in c]
    sources.sort(key = lambda c: c['mjd'])
    latestSource = sources[-1]
    latestMjd = latestSource['mjd']
    latestMag = latestSource['magpsf']
    oldSources = sources[:-1]
    mags = []
#    mjds = []
    for s in oldSources:
        if latestMjd - s['mjd'] < timeInterval:
            mags.append(s['magpsf'])
#            mjds.append(s['mjd'])
    n = len(mags)
    if len(mags) < nLimit:
        return None
    mean = sum(mags)/n
    var = sum([(m-mean)**2 for m in mags])/n
    sd   = math.sqrt(var)
    sigma = abs((latestMag - mean)/sd)
    return sigma

### Fetch the lightcurves of the found objects

In [8]:
timeInterval = 20       # days
nLimit       = 10       # must have at least this number in timeInterval
for row in results:
    objectId = row['objectId']
    alert = L.object(objectId)
    sigma = sigmaJump(alert, timeInterval, nLimit)
    if sigma and sigma > 3:
        print('https://lasair-ztf.lsst.ac.uk/objects/%s %8.3f' % (objectId, sigma))